# Autocircuit Test

In [17]:
%load_ext autoreload
%autoreload 2

import os, sys
# Add the directory containing the utils folder to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import torch as t

from auto_circuit.data import load_datasets_from_json
from auto_circuit.experiment_utils import load_tl_model
from auto_circuit.visualize import draw_seq_graph
from tqdm.notebook import tqdm

from utils.circuit_discovery import acdc_discovery, prepare_model_acdc
from utils.testing import access_wandb_runs

from utils.testing import load_our_model, download_models, update_run

from os.path import abspath
from pathlib import Path

results_key = 'acdc_ioi_edges'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [12]:
# Get the no-ablation model
all_wandb_runs = access_wandb_runs(filters=None)

baseline_run = None
for run in all_wandb_runs: # Linear search on small list
    if run.name == "no-ablation-baseline":
        baseline_run = run
        break

download_models([baseline_run], "../model_weights")

Downloaded best_model_20241014.pt to ../model_weights/no-ablation-baseline
Downloaded config.yaml to ../model_weights/no-ablation-baseline


In [13]:
baseline_model = load_our_model("../model_weights/no-ablation-baseline/", device=device)
patched_baseline_model = prepare_model_acdc(baseline_model, device)

In [14]:
path = Path("../datasets/autocircuit_ioi_customized_prompts.json")
train_loader, test_loader = load_datasets_from_json(
    model=patched_baseline_model, # Used to get tokenizer
    path=path,
    device=device,
    prepend_bos=True,
    batch_size=64, # ACDC only uses the first batch? Higher batch sizes reduce the variance of the edges discovered
    train_test_size=(128, 128),
)


## Testing autocircuit's ACDC using regular transformer

In [15]:
def test_model(patched_model, data_loader, save_path=None, score_threshold=0.03):
    total_edges, remaining_edges, pruned_scores = acdc_discovery(patched_model, data_loader, tau_bases=[score_threshold * 100])
    print(f"Total edges: {total_edges}")
    print(f"Remaining edges: {remaining_edges}")
    
    # Pruned neurons are given a value of inf
    # Interactive Image may cause performance issues
    if save_path:
        save_path = Path(save_path).resolve()

    _ = draw_seq_graph(patched_model, pruned_scores, score_threshold=score_threshold, orientation='v', layer_spacing='True', file_path=save_path)
    
    return remaining_edges, pruned_scores

In [75]:
regular_remaining_edges, regular_pruned_scores = test_model(patched_baseline_model, train_loader, save_path="../plots/our-baseline.svg")

Total edges: 23981
Remaining edges: 79


In [ ]:
results = {results_key: regular_remaining_edges}
update_run(baseline_run, results)

## Testing ACDC using ablated transformer

The loop currently only supports overall ablation models

In [3]:
wandb_ablated_runs = access_wandb_runs()
download_models(wandb_ablated_runs, "../model_weights") # Will skip downloading if the model already exists

Downloaded best_model_20241105.pt to ../model_weights/fanciful-fog-78
Downloaded config.yaml to ../model_weights/fanciful-fog-78
Downloaded best_model_20241106.pt to ../model_weights/earnest-moon-79
Downloaded config.yaml to ../model_weights/earnest-moon-79
Downloaded best_model_20241105.pt to ../model_weights/super-violet-80
Downloaded config.yaml to ../model_weights/super-violet-80
Downloaded best_model_20241109.pt to ../model_weights/upbeat-glitter-83
Downloaded config.yaml to ../model_weights/upbeat-glitter-83
Downloaded best_model_20241109.pt to ../model_weights/comfy-cherry-84
Downloaded config.yaml to ../model_weights/comfy-cherry-84
Downloaded best_model_20241111.pt to ../model_weights/cosmic-leaf-81-part2
Downloaded config.yaml to ../model_weights/cosmic-leaf-81-part2
Downloaded best_model_20241114.pt to ../model_weights/major-planet-86-part3
Downloaded config.yaml to ../model_weights/major-planet-86-part3
Downloaded best_model_20241116.pt to ../model_weights/light-morning-92


In [19]:
for run in tqdm(wandb_ablated_runs, desc="Processing runs"):
    
    if "acdc_ioi_edges" in run.summaryMetrics.keys():
        print(f"Skipping run {run.name} as it has already been processed")
        continue
    
    if run.state != "finished":
        print(f"Skipping run {run.name} as it is not finished")
        continue
    
    print(f"Processing run {run.name}")
    
    model_dir = f"../model_weights/{run.name}"
    
    try:
        ablated_trained_model = load_our_model(model_dir, device=device)
    except Exception as e:
        print(f"Failed to load model from {run.name}. Possibly due to differing state_dict keys.")
        # print(f"{e}")
        continue   
        
    patched_ablated_trained_model = prepare_model_acdc(ablated_trained_model, device)
    
    remaining_edges, pruned_scores = test_model(patched_ablated_trained_model, train_loader, save_path=f"../plots/{run.name}.svg")
    
    results = {results_key: remaining_edges}
    update_run(run, results)

Skipping run fanciful-fog-78 as it has already been processed
Skipping run earnest-moon-79 as it has already been processed
Skipping run super-violet-80 as it has already been processed
Skipping run upbeat-glitter-83 as it has already been processed
Skipping run comfy-cherry-84 as it has already been processed
Skipping run cosmic-leaf-81-part2 as it has already been processed
Skipping run major-planet-86-part3 as it has already been processed
Skipping run light-morning-92 as it has already been processed
Skipping run deft-dew-98-part2 as it has already been processed
Skipping run solar-vortex-103 as it has already been processed
Skipping run peachy-paper-104 as it has already been processed
Skipping run valiant-music-105 as it has already been processed
Processing run usual-silence-107
Total edges: 23981
Remaining edges: 54


Processing run amber-waterfall-109
Total edges: 23981
Remaining edges: 70


## Circuit Comparison

Contains code for making a comparison between the baseline and a model of our choice

In [87]:
data = {
    'Base Model': {
        'edges': regular_remaining_edges,
        'path': '../plots/our-baseline.svg',
    },
    'Ablated Model': {
        'edges': remaining_edges, # Becareful with this once we have multiple runs
        'path': '../plots/fanciful-fog-78.svg',
    }
}

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import cairosvg
import io

def save_comparison_plot(data, output_path, dpi=600):
    def load_image(path):
        # Check if file is SVG
        if path.lower().endswith('.svg'):
            # Convert SVG to PNG in memory
            png_data = cairosvg.svg2png(url=path)
            return Image.open(io.BytesIO(png_data))
        else:
            return Image.open(path)

    # Load images with SVG support
    base_img = load_image(data['Base Model']['path'])
    ablated_img = load_image(data['Ablated Model']['path'])

    # Resize with high-quality interpolation
    target_size = (1000, 1000)
    base_img_resized = base_img.resize(target_size, Image.Resampling.LANCZOS)
    ablated_img_resized = ablated_img.resize(target_size, Image.Resampling.LANCZOS)

    # Create larger figure with higher DPI
    plt.rcParams['figure.dpi'] = 300
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    # Plot images with captions
    ax1.imshow(base_img_resized)
    ax1.set_xlabel(f'Base Model\n({data["Base Model"]["edges"]} edges)')
    ax1.set_xticks([])
    ax1.set_yticks([])

    ax2.imshow(ablated_img_resized)
    ax2.set_xlabel(f'Model trained with Self-Ablation\n({data["Ablated Model"]["edges"]} edges)')
    ax2.set_xticks([])
    ax2.set_yticks([])

    # Set overall title
    fig.suptitle('IOI Circuit Comparison Across Tiny Stories Models (τ=0.03)', fontsize=14, y=1)

    # Save based on file extension
    plt.tight_layout()
    ext = os.path.splitext(output_path)[1].lower()
    
    if ext == '.svg':
        plt.savefig(output_path, format='svg', bbox_inches='tight')
    else:
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight')
    
    plt.close()
    
save_comparison_plot(data, '../plots/ioi-circuit-comparison.png')